In [ ]:
# Author: Aaron Valoroso
# Date: June 13th, 2018
# Topic: WMTS and image stitching

from PIL import Image #conda install -c anaconda pillow 
import os
import sys
import math
import requests
import rasterio
import numpy as np
from rasterio.io import MemoryFile
from io import BytesIO
from itertools import product
from ipywidgets import widgets, HBox, Label# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [ ]:
# Get input data

style = {'description_width': 'initial'}

zoom_widget_value = widgets.IntSlider(min=0,max=19,step=1,description='Zoom Level:',orientation='horizontal')
display(zoom_widget_value)

print("Upper left hand corner of bbox.")
long_1_widget_value = widgets.FloatText(description='Longitude-1:')
display(long_1_widget_value)

lat_1_widget_value = widgets.FloatText(description='Latitude-1:')
display(lat_1_widget_value)

print("Bottom right hand corner of bbox.")
long_2_widget_value = widgets.FloatText(description='Longitude-2:')
display(long_2_widget_value)

lat_2_widget_value = widgets.FloatText(description='Latitude-2:')
display(lat_2_widget_value)

In [ ]:
# Main
%time
# zoom_level = zoom_widget_value.value
# long_1 = long_1_widget_value.value
# lat_1 = lat_1_widget_value.value
# long_2 = long_2_widget_value.value
# lat_2 = lat_2_widget_value.value
zoom_level = 15
long_1 = -78.997643
lat_1 = 40.382589
long_2 = -78.735128
lat_2 = 40.183469
url = 'http://c.tile.openstreetmap.org/{0}/{1}/{2}.png'

combined_latitudes_list = [lat_1, lat_2]
convereted_lat_tile_list = [int(1/(2*math.pi)*2**zoom_level*(math.pi-math.log(math.tan(math.pi/4+math.radians(y)/2)))) for y in combined_latitudes_list]
print("Conv-Lat: ", convereted_lat_tile_list)

combined_longitudes_list = [long_1, long_2]
convereted_long_tile_list = [int(2**(zoom_level-1)*(x/180+1)) for x in combined_longitudes_list]
print("Conv-Long: ", convereted_long_tile_list)

first_lat = math.degrees(2*(math.atan(math.exp(math.pi-(convereted_long_tile_list[0]*2*math.pi)/2**zoom_level))-math.pi/4))
second_lat = math.degrees(2*(math.atan(math.exp(math.pi-((convereted_long_tile_list[-1] + 1)*2*math.pi)/2**zoom_level))-math.pi/4))
print("First_Lat: ", first_lat)
print("Second_Lat: ", second_lat)

first_long = lon = math.degrees(convereted_lat_tile_list[0]*2*math.pi/(2**zoom_level) - math.pi)
second_long = lon = math.degrees((convereted_lat_tile_list[-1] + 1)*2*math.pi/(2**zoom_level) - math.pi)
print("First_Long: ", first_long)
print("Second_Long: ", second_long)
number_of_y_tiles = convereted_lat_tile_list[-1] - convereted_lat_tile_list[0]
# if number_of_y_tiles <= 0:
#     number_of_y_tiles = 1
print("Min-y: ", number_of_y_tiles)

number_of_x_tiles = convereted_long_tile_list[-1] - convereted_long_tile_list[0]
# if number_of_x_tiles <= 0:
#     number_of_x_tiles = 1
print("Min-x: ", number_of_x_tiles)

lat_range = range(convereted_lat_tile_list[0], (convereted_lat_tile_list[1] + 1))
long_range = range(convereted_long_tile_list[0], (convereted_long_tile_list[1] + 1))
print("Lat-R: ", lat_range)
print("Long-R: ", long_range)

print("Max-x: ", (number_of_x_tiles + 1) * 256)
print("Max-y: ", (number_of_y_tiles + 1) * 256)

loop_itereator = 0
stitched_image = Image.new('RGB', ((number_of_x_tiles + 1) * 256, (number_of_y_tiles + 1) * 256 ))
for x, y in product(long_range, lat_range):
    response = requests.get(url.format(zoom_level, x, y), verify=False)
    if response.status_code == 200:
        picture_contents = response.content
    print(x, " ", y)
    if loop_itereator == 0:
        incoming_image = Image.open(BytesIO(picture_contents))
        min_tile_index_width = x
        min_tile_index_height = y
        x = (x - min_tile_index_width) * 256
        y = (y - min_tile_index_height) * 256
        stitched_image.paste(im=incoming_image, box=(x, y))
        incoming_image.close()
        loop_itereator += 1
    else:
        incoming_image = Image.open(BytesIO(picture_contents))
        x = (x - min_tile_index_width) * 256
        y = (y - min_tile_index_height) * 256
        stitched_image.paste(im=incoming_image, box=(x, y))
        incoming_image.close()

        
stitched_image.show()      
file_path = '/Users/rditlaav/Documents/GitHub/Python/my_image2.tiff'
Z = np.array(stitched_image)
transform = rasterio.transform.from_bounds(first_long, second_lat, second_long, first_lat, (number_of_x_tiles + 1) * 256, (number_of_y_tiles + 1) * 256)
Z = np.moveaxis(Z, -1, 0)
with rasterio.open(file_path, 'w', driver='GTiff', height=Z.shape[1], 
                width=Z.shape[2], count=Z.shape[0], dtype=Z.dtype,
                crs='EPSG:3857', transform=transform) as dst:
    dst.write(Z, indexes=[1, 2, 3])
stitched_image.close()
